In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import keras
import datetime
from tqdm import tqdm
from sklearn.model_selection import TimeSeriesSplit
import matplotlib.pyplot as plt

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
#This notebooks is a work in progress and incomplete.

In [3]:
train=pd.read_csv("/kaggle/input/playground-series-s3e19/train.csv",index_col='id')
test=pd.read_csv("/kaggle/input/playground-series-s3e19/test.csv",index_col='id')

In [4]:
train.head()

,date,country,store,product,num_sold
id,,,,,
0,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Improve Your Coding,63
1,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Train More LLMs,66
2,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win Friends and Influence People,9
3,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win More Kaggle Competitions,59
4,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Write Better,49


In [5]:
len(train['date'].unique())

1826

In [6]:
cat_features=['country','store','product']
for feature in cat_features:
    print(feature,':',train[feature].unique())

country : ['Argentina' 'Canada' 'Estonia' 'Japan' 'Spain']
store : ['Kaggle Learn' 'Kaggle Store' 'Kagglazon']
product : ['Using LLMs to Improve Your Coding' 'Using LLMs to Train More LLMs'
 'Using LLMs to Win Friends and Influence People'
 'Using LLMs to Win More Kaggle Competitions' 'Using LLMs to Write Better']


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136950 entries, 0 to 136949
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   date      136950 non-null  object
 1   country   136950 non-null  object
 2   store     136950 non-null  object
 3   product   136950 non-null  object
 4   num_sold  136950 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 6.3+ MB


In [8]:
def data_encoding(dataset):
    cat_features=['country','store','product']
    for feature in tqdm(cat_features):
        dictionary={}
        n=1
        print('feature:',feature)
        print('categories:',dataset[feature].unique())
        for sub_cat in dataset[feature].unique():
            dictionary[sub_cat]=n
            n+=1
        print('encoding rule:',dictionary)
        dataset[feature]=dataset[feature].map(dictionary)
        dummies=pd.get_dummies(dataset[feature],prefix=feature)
        for key in dummies:
            dataset[key]=dummies[key]
        dataset=dataset.drop(feature,axis=1)
    dataset['date']=pd.to_datetime(dataset['date'],format='%Y-%m-%d')
    print("Dates converted to datetime64 format")
    return dataset

In [9]:
train=data_encoding(train)
test=data_encoding(test)

  0%|          | 0/3 [00:00<?, ?it/s]

feature: country
categories: ['Argentina' 'Canada' 'Estonia' 'Japan' 'Spain']
encoding rule: {'Argentina': 1, 'Canada': 2, 'Estonia': 3, 'Japan': 4, 'Spain': 5}


 67%|██████▋   | 2/3 [00:00<00:00, 19.86it/s]

feature: store
categories: ['Kaggle Learn' 'Kaggle Store' 'Kagglazon']
encoding rule: {'Kaggle Learn': 1, 'Kaggle Store': 2, 'Kagglazon': 3}
feature: product
categories: ['Using LLMs to Improve Your Coding' 'Using LLMs to Train More LLMs'
 'Using LLMs to Win Friends and Influence People'
 'Using LLMs to Win More Kaggle Competitions' 'Using LLMs to Write Better']
encoding rule: {'Using LLMs to Improve Your Coding': 1, 'Using LLMs to Train More LLMs': 2, 'Using LLMs to Win Friends and Influence People': 3, 'Using LLMs to Win More Kaggle Competitions': 4, 'Using LLMs to Write Better': 5}


100%|██████████| 3/3 [00:00<00:00, 18.33it/s]


Dates converted to datetime64 format


  0%|          | 0/3 [00:00<?, ?it/s]

feature: country
categories: ['Argentina' 'Canada' 'Estonia' 'Japan' 'Spain']
encoding rule: {'Argentina': 1, 'Canada': 2, 'Estonia': 3, 'Japan': 4, 'Spain': 5}
feature: store


100%|██████████| 3/3 [00:00<00:00, 68.65it/s]

categories: ['Kaggle Learn' 'Kaggle Store' 'Kagglazon']
encoding rule: {'Kaggle Learn': 1, 'Kaggle Store': 2, 'Kagglazon': 3}
feature: product
categories: ['Using LLMs to Improve Your Coding' 'Using LLMs to Train More LLMs'
 'Using LLMs to Win Friends and Influence People'
 'Using LLMs to Win More Kaggle Competitions' 'Using LLMs to Write Better']
encoding rule: {'Using LLMs to Improve Your Coding': 1, 'Using LLMs to Train More LLMs': 2, 'Using LLMs to Win Friends and Influence People': 3, 'Using LLMs to Win More Kaggle Competitions': 4, 'Using LLMs to Write Better': 5}
Dates converted to datetime64 format


In [10]:
train.head()

,date,num_sold,country_1,country_2,country_3,country_4,country_5,store_1,store_2,store_3,product_1,product_2,product_3,product_4,product_5
id,,,,,,,,,,,,,,,
0,2017-01-01,63,1,0,0,0,0,1,0,0,1,0,0,0,0
1,2017-01-01,66,1,0,0,0,0,1,0,0,0,1,0,0,0
2,2017-01-01,9,1,0,0,0,0,1,0,0,0,0,1,0,0
3,2017-01-01,59,1,0,0,0,0,1,0,0,0,0,0,1,0
4,2017-01-01,49,1,0,0,0,0,1,0,0,0,0,0,0,1


In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136950 entries, 0 to 136949
Data columns (total 15 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   date       136950 non-null  datetime64[ns]
 1   num_sold   136950 non-null  int64         
 2   country_1  136950 non-null  uint8         
 3   country_2  136950 non-null  uint8         
 4   country_3  136950 non-null  uint8         
 5   country_4  136950 non-null  uint8         
 6   country_5  136950 non-null  uint8         
 7   store_1    136950 non-null  uint8         
 8   store_2    136950 non-null  uint8         
 9   store_3    136950 non-null  uint8         
 10  product_1  136950 non-null  uint8         
 11  product_2  136950 non-null  uint8         
 12  product_3  136950 non-null  uint8         
 13  product_4  136950 non-null  uint8         
 14  product_5  136950 non-null  uint8         
dtypes: datetime64[ns](1), int64(1), uint8(13)
memory usage: 4.8 MB


In [12]:
def datetime_features(df_temp):
    """
ref:-https://www.kaggle.com/code/chingiznurzhanov/timeseriessplit-catboost-trick
    """
    df_temp['month'] = df_temp['date'].dt.month
    df_temp['day'] = df_temp['date'].dt.day
    df_temp['year'] = df_temp['date'].dt.year
    df_temp['dayofweek'] = df_temp['date'].dt.dayofweek
    df_temp['quarter'] = df_temp['date'].dt.quarter
    df_temp['dayofmonth'] = df_temp['date'].dt.day
    df_temp['weekofyear'] = df_temp['date'].dt.weekofyear
    df_temp['is_month_start'] = df_temp.date.dt.is_month_start.astype(np.int8)
    df_temp['is_month_end'] = df_temp.date.dt.is_month_end.astype(np.int8)
    df_temp['monday'] = df_temp.date.dt.weekday.eq(0).astype(np.uint8)
    df_temp['tuesday'] = df_temp.date.dt.weekday.eq(1).astype(np.uint8)
    df_temp['wednesday'] = df_temp.date.dt.weekday.eq(2).astype(np.uint8)
    df_temp['thursday'] = df_temp.date.dt.weekday.eq(3).astype(np.uint8)
    df_temp['friday'] = df_temp.date.dt.weekday.eq(4).astype(np.uint8)
    df_temp['saturday'] = df_temp.date.dt.weekday.eq(5).astype(np.uint8)
    df_temp['sunday'] = df_temp.date.dt.weekday.eq(6).astype(np.uint8)
    df_temp['is_quarter_end'] = df_temp['date'].dt.is_quarter_end
    df_temp['is_quarter_start'] = df_temp['date'].dt.is_quarter_start
    df_temp['is_year_end'] = df_temp['date'].dt.is_year_end
    df_temp['is_year_start'] = df_temp['date'].dt.is_year_start
    return df_temp

In [13]:
#splitting into training and validation sets
val=train[round(0.7*len(train)):]
train=train[:round(0.7*len(train))]

In [14]:
#tf.convert_to_tensor(train)